# Задачи по SQL

## Задача 1

Формулировка: вывести название и цену для всех анализов, которые продавались 5 февраля 2020 и всю следующую неделю.

```sql
SELECT a.an_name, a.an_price
FROM Analysis a
JOIN Orders o ON a.an_id = o.ord_an
WHERE o.ord_datetime >= '2020-02-05 00:00:00'
  AND o.ord_datetime < '2020-02-12 00:00:00';
```

## Задача 2
Формулировка: нарастающим итогом рассчитать, как увеличивалось количество проданных тестов каждый месяц каждого года с разбивкой по группе.

```sql
WITH MonthlySales AS (
    SELECT 
        EXTRACT(YEAR FROM o.ord_datetime) AS year,
        EXTRACT(MONTH FROM o.ord_datetime) AS month,
        a.an_group AS group_name,
        COUNT(o.ord_id) AS total_sold
    FROM 
        Orders o
    JOIN 
        Analysis a ON o.ord_an = a.an_id
    GROUP BY 
        year, month, group_name
),
CumulativeSales AS (
    SELECT 
        year,
        month,
        group_name,
        total_sold,
        SUM(total_sold) OVER (PARTITION BY group_name ORDER BY year, month) AS cumulative_total
    FROM 
        MonthlySales
)
SELECT 
    year,
    month,
    group_name,
    total_sold,
    cumulative_total
FROM 
    CumulativeSales
ORDER BY 
    group_name, year, month;
```

## Задача 3
Формулировка: в данной таблице в какой-то момент времени появились полные дубли. Предложите способ для избавления от них без создания новой таблицы.
```sql
WITH Duplicates AS (
    SELECT 
        client_id,
        client_name,
        client_balance_date,
        client_balance_value,
        ROW_NUMBER() OVER (PARTITION BY client_id, client_balance_date ORDER BY client_balance_date) AS row_num
    FROM 
        ClientBalance
)
DELETE FROM ClientBalance
WHERE (client_id, client_balance_date) IN (
    SELECT client_id, client_balance_date
    FROM Duplicates
    WHERE row_num > 1
);
```